* **Step 1: Import libraries**
* **Step 2: Create a list for parameters**
* **Step 3: Append the lists**
* **Step 4: Loop through the links &rarr; get 30 links**

In [21]:
from bs4 import BeautifulSoup
from selenium import webdriver
import json
import requests
import time
import re
import pandas as pd

driver = webdriver.Chrome()

location = []
property_type = []
property_subtype = []
price = []
sale_type = []
number_of_bedrooms =[]
living_area = []

equipped_kitchen = []
furnished = []
open_fire = []
terrace = []
terrace_area = []
garden = []

garden_area = []
surface_area_land = []
number_of_facades  = []
pool = []
#condition = []

def get_data_from_url(link):
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, "lxml")
    
    div_with_script = soup.find("div", attrs={"class": "classified"})
    script_text = div_with_script.script.text.split("= ", 1)[1]
    json_data = json.loads(script_text.rstrip()[:-1])

    
    property_data = json_data["property"]
    price_data = json_data["price"]
    transaction_data = json_data["transaction"]
    
    #location
    town = property_data["location"]["locality"]
    postcode = property_data["location"]["postalCode"]
    location.append((town,postcode) if property_data["location"] != None else "N/A")
    
    #Type of property (House/apartment/etc)
    property_type.append(property_data["type"])
    
    #Subtype of property (Bungalow, Chalet, Mansion, ...)
    property_subtype.append(property_data["subtype"])
    
    #Price
    price.append(price_data["mainValue"])
    
    #Type of sale (Exclusion of life sales)
    sale_type.append(price_data["type"])
    
    #Number of rooms
    number_of_bedrooms.append(property_data["bedroomCount"])
    
    #Livable space m2
    living_area.append(property_data["netHabitableSurface"])
    
    #Fully equipped kitchen(Y/N)
    """"if (property_data['kitchen']["type"]) == "Not installed":
        equipped_kitchen.append("No")
    elif (property_data['kitchen']["type"]) == None:
        equipped_kitchen.append("N/A")
    else:
        equipped_kitchen.append("Yes")
        #equipped_kitchen.append(property_data['kitchen']["type"])"""
    equipped_kitchen.append(property_data['kitchen']["type"] if property_data["kitchen"] != None else "Unknown")
    
    #Furnished or not
    if transaction_data["sale"]["isFurnished"] == True:
        furnished.append("Yes")
    elif transaction_data["sale"]["isFurnished"] == False:
        furnished.append("No")
    else:
        furnished.append("N/A")
    
    #Open fire
    if property_data["fireplaceExists"] == True:
        open_fire.append("Yes")
    elif property_data["fireplaceExists"] == False:
        open_fire.append("No")
    else: 
        open_fire.append("N/A")   
    
    #Terrace (Yes/No) If yes: Area
    if property_data["hasTerrace"] != None:
        terrace.append("Yes,")
        #terrace.append(property_data["terraceSurface"])
    elif property_data["hasTerrace"] == None:
        terrace.append("No")
    else: 
        terrace.append("N/A")

    #Garden (Yes/No) If yes: Area
    if property_data["hasGarden"] == True:
        garden.append("Yes")
        #garden.append(property_data["gardenSurface"])
    elif property_data["hasTerrace"] == None:
        garden.append("No")
    else: 
        garden.append("N/A")
    
    #Surface of the land
    if property_data["land"] == None:
        surface_area_land.append("None")
    elif property_data["land"] != None:
        surface_area_land.append(property_data["land"]["surface"])
    else: 
        surface_area_land.append("N/A")                           
    
    #Number of facades
    number_of_facades.append(property_data["building"]["facadeCount"] if property_data["building"] != None else "N/A")
                                
    #Swimming pool (Yes/No)
    if property_data["hasSwimmingPool"] == True:
        pool.append("Yes")
    elif property_data["hasSwimmingPool"] != True:
        pool.append("No")
                                
    #State of the building (New, to be renovated, ...)
    #None type is not callable(to be continued)
    #condition.append(property_data["building"]["condition"] if property_data["building"]["condition"] != None else "N/A")
    #condition.append(property_data["building"]["condition"] if property_data["building"]["condition"] != None else "Unknown")

                                
    df = pd.DataFrame()
    df["location"] = location
    df["property_type"] = property_type
    df["property_subtype"] = property_type                        
    df["price"] = price
    df["bedrooms"] = number_of_bedrooms
    df["living_area"] = living_area
    df["equipped_kitchen"] = equipped_kitchen
    df["furnished"] = furnished
    df["open_fire"] = open_fire
    df["terrace"] = terrace
    df["garden"] = garden
    df["surface_area_land"] = surface_area_land
    df["facades"] = number_of_facades
    df["swimming_pool"] = pool
    #df["condition"] = condition

    df.to_csv("data.csv", mode='w', header=True)     

                                
time.sleep(3)

counter = 0

links = []

#Only gets links from 1 page, needs to be 334 pages to be 10k links
for i in range(1,2):

    url = f"https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&page={i}&orderBy=relevance"
    print("\n", url, "\n")
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "lxml")

    for html_link in soup.find_all("li", {"class":"search-results__item"}):
        for link in html_link.find_all("a", {"class":"card__title-link"}):
            counter += 1
            print(f"Page number: {i}, Link number: {counter}")
            print(link["href"])
            links.append(link["href"])
            get_data_from_url(link["href"])


 https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&page=1&orderBy=relevance 

Page number: 1, Link number: 1
https://www.immoweb.be/en/classified/flat-studio/for-sale/brussels-city/1000/9779551?searchId=621f76ff6f564
Page number: 1, Link number: 2
https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/arendonk/2370/9781074?searchId=621f76ff6f564
Page number: 1, Link number: 3
https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/anderlecht/1070/9780840?searchId=621f76ff6f564
Page number: 1, Link number: 4
https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/evere/1140/9780814?searchId=621f76ff6f564
Page number: 1, Link number: 5
https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/evere/1140/9779662?searchId=621f76ff6f564
Page number: 1, Link number: 6
https://www.immoweb.be/en/classified/mansion/for-sale/ronse/9600/9781201?searchId=621f76ff6f564
Page n